# Exploring MIRI cubes

In [ ]:
from jwst import datamodels
import matplotlib.pyplot as plt
import reproject

In [ ]:
d = datamodels.IFUCubeModel("/Users/dvandeputte/Projects/PDR ERS/simulated-data-stsci/MRS2/stage3/Level3_ch1-long_s3d.fits")
d

In [ ]:
d.info(max_rows=None)

In [ ]:
d.meta.instrument.instance

In [ ]:
d.shape

## Examine wcs and wavelengths

In [ ]:
type(d.wcs)
d.wcs

In [ ]:
d.wcs.pixel_to_world(10, 10, 10)

In [ ]:
wavs = d.wcs.pixel_to_world(10, 20, range(0, d.shape[0]))
wavs

In [ ]:
d.wavetable

wavetable seems to be empty, but we can get a list of wavelengths this way

In [ ]:
d.wcs.spectral.pixel_to_world(range(0, d.shape[0]))

In [ ]:
wavs[1].value

## plotting slices

In [ ]:
ax = plt.subplot(projection=d.wcs, slices=('x', 'y', 10))
ax.imshow(d.data[10])


## workspace for making plots
Avoid having to run the reprojection over and over by using this code, and then developing some plots below.

In [ ]:
from merge_miri_cubes import get_miri_cubes
from resample_and_merge_spitzer_cubes import reproject_cube_data
import wcshacks

In [ ]:
nx, ny = 5, 5
newwcs = wcshacks.make_ra_dec_wcs(0, 0, 1 / 3600, nx, ny)
cubes = get_miri_cubes("../simulated-data-stsci/MRS2/stage3/", "Level3")

In [ ]:
rpj_data = [
        wcshacks.reproject_cube_data(c.data, c.wcs.sub((1, 2)), newwcs, ny, nx)
        for c in cubes
    ]

In [ ]:
titles = ["ch"
        + cube_model.meta.instrument.channel
        + " "
        + cube_model.meta.instrument.band
        for cube_model in cubes
]
titles

In [ ]:
def plot12cubes(cbs, titles):
    fig, axs = plt.subplots(4,3)
    for ax, dat, title in zip(axs.flatten(), cbs, titles):
        ax.imshow(dat[len(dat) // 2])
        ax.set_title(title)
    fig.set_size_inches(6,8)
    fig.tight_layout()
    return fig, axs

In [ ]:
fig, axs = plot12cubes([c.data for c in cubes], titles)
fig.savefig('miri_cubes.pdf')

In [ ]:
fig, axs = plot12cubes(rpj_data, titles)
fig.savefig('miri_cubes_rpj.pdf')